In [1]:
import pandas as pd
import os

from bokeh.plotting import figure
from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot

output_notebook()

os.chdir("/home/tales/dev/master/mdc_analysis/")
print("working dir", os.getcwd())

from src.dao import csv_dao
from src.plot import plot
from src.utils import geo
from src.entity.geo_circle import GeoCircle
from src.utils.color_utils import palette
from src.poi_grabber.google_places import credentials, nearby_search, next_page, group_data_result, save_data, make_request
from src.poi_grabber import google_places

import gc

Loading BokehJS ...

working dir /home/tales/dev/master/mdc_analysis


In [3]:
# rc50 = csv_dao.load_request_circles_df(50)
rc75 = csv_dao.load_request_circles_df(75)
# rc200 = csv_dao.load_request_circles_df(200)
# rc300 = csv_dao.load_request_circles_df(300)
# rc400 = csv_dao.load_request_circles_df(400)
# rc500 = csv_dao.load_request_circles_df(500)
# rc600 = csv_dao.load_request_circles_df(600)
# rc700 = csv_dao.load_request_circles_df(700)
# rc800 = csv_dao.load_request_circles_df(800)
# rc1k = csv_dao.load_request_circles_df(1000)

print(len(rc75))
rc75.head()

12291


,rc_id,latitude,longitude,radius_m,search_tolerance,sr_ids
0,46.45373,46.45373,6.86650,75,0.0006,"[6189_0, 6189_0, 6189_1, 6175_129, 6175_13, 61..."
1,46.51158,46.51158,6.66102,75,0.0006,"[6189_10, 6189_10, 6177_2512, 6177_2646]"
2,46.54204,46.54204,6.64408,75,0.0006,"[6189_100, 6189_100, 6189_137, 6189_138, 6189_..."
3,46.54100,46.54100,6.64300,75,0.0006,"[6189_101, 6189_101]"
4,46.53000,46.53000,6.65000,75,0.0006,"[6189_102, 6189_102]"


In [4]:
def request_circle_size(request_circles):
    return request_circles["request_circle"].apply(lambda rc : len(rc.get_data()))

def valid_pois(pois):
    return pois[pois["amenity"].isna() == False]

def pois_in_request_circle(request_circles, pois):
    pois["latitude"] = pois["lat_4326"]
    pois["longitude"] = pois["lon_4326"]
    rc_geo_circles = request_circles.apply(lambda row : GeoCircle(row["latitude"], row["longitude"], radius_m=row["radius_m"], id=row["rc_id"], searching_tolerance=row["search_tolerance"]), axis=1).tolist()
    
    for rc in rc_geo_circles:
        pois_in = []
        use_pois = geo.slice_geo_data(pois, center=rc, search_tolerance=rc.searching_tolerance)

        for index, poi in use_pois.iterrows():
            if rc.contains_point(latitude=poi["latitude"], longitude=poi["longitude"]):
                pois_in.append(poi.to_dict())
        
        rc.put_data(pd.DataFrame(pois_in))
    return rc_geo_circles

## Putting HOT OSM POIs inside Request Circle

In [ ]:
hotosm_pois = valid_pois(csv_dao.load_hot_osm_pois())

In [ ]:
rc75_with_pois_inside = pd.DataFrame({"request_circle" : pois_in_request_circle(rc75, hotosm_pois)})
rc75_with_pois_inside["size"] = request_circle_size(rc75_with_pois_inside)
rc75_with_pois_inside=rc75_with_pois_inside.sort_values(by="size")
len(rc75_with_pois_inside), len(rc75_with_pois_inside[rc75_with_pois_inside["size"] == 0])

In [5]:
rc75.head()

,rc_id,latitude,longitude,radius_m,search_tolerance,sr_ids
0,46.45373,46.45373,6.86650,75,0.0006,"[6189_0, 6189_0, 6189_1, 6175_129, 6175_13, 61..."
1,46.51158,46.51158,6.66102,75,0.0006,"[6189_10, 6189_10, 6177_2512, 6177_2646]"
2,46.54204,46.54204,6.64408,75,0.0006,"[6189_100, 6189_100, 6189_137, 6189_138, 6189_..."
3,46.54100,46.54100,6.64300,75,0.0006,"[6189_101, 6189_101]"
4,46.53000,46.53000,6.65000,75,0.0006,"[6189_102, 6189_102]"


In [ ]:
(rc75_with_pois_inside["size"] == 0).sum(), len(rc75_with_pois_inside)

In [ ]:
already_requested = google_places.load_all_request_metadata(75)
already_requested.sample(6)

In [ ]:
rc75_with_pois_inside["rc_id"] = rc75_with_pois_inside.apply(lambda row : row["request_circle"].id, axis=1)

In [ ]:
use_rc75_with_pois_inside = rc75_with_pois_inside[~rc75_with_pois_inside["rc_id"].isin(already_requested["rc_id"])]

print(len(use_rc75_with_pois_inside), "requests to do")

for index, rc_row in use_rc75_with_pois_inside.iterrows():    
    print(rc_row["rc_id"])
    rc = rc_row["request_circle"]
    data_pages, n_requests = make_request(rc)
    data = group_data_result(data_pages)
    save_data(data, rc, n_requests)

In [ ]:
all_metadata = google_places.load_all_request_metadata(75)

In [ ]:
all_circle_data = google_places.load_all_request_circle_data(75)

In [ ]:
all_circle_data[0].head()

In [ ]:
all_circle_data[0].to_csv('../google-places-poi-grabber/data/request_circle_75.csv'.format(75), index=False)

In [6]:
import json
RAW_DATA_REQUESTS_DIR = '../google-places-poi-grabber/data/raw_data_{}/'
def load_request_result_single_file(radius_m, filename):
    with open(RAW_DATA_REQUESTS_DIR.format(radius_m) + filename) as json_file:
        return json.load(json_file)

In [7]:
meta = google_places.load_all_request_metadata(radius_m=75)
print(len(meta))
meta.head()

12315


,data_filename,latitude,longitude,n_requests,radius_m,rc_id
0,results_rc_5993_2498.json,46.55695,6.61832,1,75,rc_5993_2498
1,results_rc_5968_207.json,46.52127,6.63633,5,75,rc_5968_207
2,results_rc_6104_99.json,46.52847,6.56697,1,75,rc_6104_99
3,results_rc_5970_96.json,46.13447,7.31627,1,75,rc_5970_96
4,results_rc_6188_432.json,46.48018,7.14446,1,75,rc_6188_432


In [6]:
empty_files = []
empty_files_meta = []

counter = 0
for filename in meta["data_filename"].tolist():
    counter += 1
    result = google_places.load_request_result_single_file(75, filename)
    if len(result) == 0:
        empty_files.append(filename)
        empty_files_meta.append(filename.replace("results", "metadata"))
    
len(empty_files)

1705

In [5]:
for filename in empty_files_meta[0:800]:
    print(filename)
    meta_file = load_request_result_single_file(75, filename)
    
    gc = GeoCircle(center_lat=meta_file["latitude"], center_lon=meta_file["longitude"], 
                   radius_m=meta_file["radius_m"], id=meta_file["rc_id"])
    
    data_pages, n_requests = make_request(gc)
    data = group_data_result(data_pages)
    save_data(data, gc, n_requests)

metadata_rc_5949_540.json
metadata_rc_5925_1765.json
metadata_rc_6041_88.json
metadata_rc_6094_652.json
metadata_rc_5950_54.json
metadata_rc_6109_165.json
metadata_rc_6070_21.json
metadata_rc_6109_815.json
metadata_rc_6014_410.json
metadata_rc_6014_122.json
metadata_rc_5989_28.json
metadata_rc_6014_584.json
metadata_rc_5925_73.json
metadata_rc_6041_52.json
metadata_rc_6045_261.json
metadata_rc_5988_191.json
metadata_rc_6094_301.json
metadata_rc_6096_131.json
metadata_rc_6109_1127.json
metadata_rc_6109_855.json
metadata_rc_6104_53.json
metadata_rc_6001_1100.json
metadata_rc_6079_165.json
metadata_rc_5949_94.json
metadata_rc_6042_416.json
metadata_rc_5986_10.json
metadata_rc_6015_152.json
metadata_rc_5988_605.json
metadata_rc_6109_934.json
metadata_rc_5979_371.json
metadata_rc_5962_42.json
metadata_rc_6043_253.json
metadata_rc_6063_215.json
metadata_rc_6036_830.json
metadata_rc_5944_186.json
metadata_rc_6042_1004.json
metadata_rc_6109_166.json
metadata_rc_6042_420.json
metadata_rc_6015_1

KeyboardInterrupt: 

In [49]:
def group_file_to_one_df():
    dfs = []
    use_df = pd.DataFrame()
    
    counter = 0
    for filename in meta["data_filename"].tolist():
        counter += 1
        if counter % 100 == 0:
            print(counter)
            
        if counter % 500 == 0:
            dfs.append(use_df)
            use_df = pd.DataFrame()
            
        result = google_places.load_request_result_single_file(75, filename)
        use_df = use_df.append(result)
    dfs.append(use_df)

    return dfs

In [50]:
dfs = group_file_to_one_df()
len(dfs)

/home/tales/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300


25

In [52]:
one_df = pd.DataFrame()
for df in dfs:
    one_df = one_df.append(df)


In [53]:
print(len(one_df))
print(len(one_df.drop_duplicates(subset="place_id")))

188280
76965


In [48]:
print(len(one_df))
print(len(one_df.drop_duplicates(subset="place_id")))

179222
75429


In [55]:
one_df.drop_duplicates(subset="place_id").to_csv('../google-places-poi-grabber/data/pois_data_75.csv', index=False)